In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load uploaded files
# Paths updated to Kaggle input directory format
df_test = pd.read_csv('/kaggle/input/test-sample/train.csv')
df_val = pd.read_csv('/kaggle/input/test-sample/val.csv')
df_combined = pd.concat([df_test, df_val], ignore_index=True)
submission_df = pd.read_csv('/kaggle/input/test-sample1/sample_submission.csv')

# Drop unnecessary columns
df_combined = df_combined.drop(columns=['team_name', 'shortname', 'bike_name', 'circuit_name','rider_name'])
df_combined['Penalty'] = df_combined['Penalty'].fillna('None')

# Normalize Unique ID and Rider Name columns
df_combined['Unique ID'] = df_combined['Unique ID'].astype(str).str.strip().str.upper()
#df['rider_name'] = df['rider_name'].astype(str).str.strip().str.lower()
submission_df['Unique ID'] = submission_df['Unique ID'].astype(str).str.strip().str.upper()
#submission_df['rider_name'] = submission_df['rider_name'].astype(str).str.strip().str.lower()

# Auto-detect the lap time column name if not explicitly available
possible_lap_time_columns = [col for col in df_combined.columns if 'lap' in col.lower() and 'time' in col.lower()]
if possible_lap_time_columns:
    lap_time_column = possible_lap_time_columns[0]  # Choose the most relevant match
    print(f"Detected lap time column: {lap_time_column}")

    # Merge on rider_name to find matching lap times
    merged_df = pd.merge(submission_df, df_combined[[ 'Unique ID', lap_time_column]] , how='left')

    # Drop duplicates if any
    merged_df = merged_df.drop_duplicates(subset=['Unique ID'])

    # Select required columns
    lap_times_from_val = merged_df[['Unique ID', lap_time_column]]

    # Print the results
    print("\nLap times and Unique IDs from validation set:")
    print(lap_times_from_val.head())

    # Save to CSV
    lap_times_from_val.to_csv('solution.csv', index=False)
    print("\nSaved lap times with Unique IDs to 'solution.csv'")
else:
    print("❌ Could not detect a lap time column automatically. Please check your dataset.")

Detected lap time column: Lap_Time_Seconds

Lap times and Unique IDs from validation set:
  Unique ID  Lap_Time_Seconds
0    288307            70.667
1    704288           103.497
2    951491            78.400
3   2591721            81.338
4   1202653           108.626

Saved lap times with Unique IDs to 'solution.csv'
